In [1]:
import os 
os.chdir("/nas/ucb/oliveradk/diverse-gen/")

In [2]:
# imports from cc experiments
import json
from functools import partial
from itertools import product
from typing import Optional, Literal, Callable
from tqdm import tqdm
from collections import defaultdict
from dataclasses import dataclass
from pathlib import Path
from copy import deepcopy
from datetime import datetime

import submitit
from submitit.core.utils import CommandFunction
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from losses.loss_types import LossType
from utils.exp_utils import get_executor, get_executor_local, run_experiments
from utils.utils import conf_to_args

In [3]:
EXP_DIR = "output/normal_data_exps/"
SCRIPT_PATH = "experiments/250124_normal_data/0_normal_data.py"

# Configs

In [5]:
# shared configs
base_config = {
    "batch_size": 32,
    "epochs": 3,
    "learning_rate": 2e-5
}

# seeds
seeds = [1, 2, 3]

# methods 
methods = {
    "DivDis": {"loss_type": LossType.DIVDIS, "aux_weight": 1.0},
    "TopK 0.1": {"loss_type": LossType.TOPK, "mix_rate_lower_bound": 0.1, "aux_weight": 1.0}, 
    "TopK 0.5": {"loss_type": LossType.TOPK, "mix_rate_lower_bound": 0.5, "aux_weight": 1.0}, 
    "ERM": {"loss_type": LossType.ERM, "aux_weight": 0.0}
}

# generate experiment configs
experiment_configs = []
for seed in seeds:
    for method_name, method_config in methods.items():
        config = base_config.copy()
        config.update(method_config)
        config["seed"] = seed
        config["exp_dir"] = f"output/normal_data/{method_name.lower().replace(' ', '_')}/seed_{seed}"
        experiment_configs.append(config)

# Run Experiments

In [ ]:
# run experiments
executor = get_executor(out_dir=EXP_DIR)
jobs = run_experiments(executor, SCRIPT_PATH, experiment_configs)